In [16]:
import numpy as np

Here we experiment with the transition of the i-th MIDI pitch the weighted combination of MIDI-pitches

Transformation matrix: 

In [17]:
T = np.load('TransitionMatrix.npy')

In [18]:
T.shape

(128, 128)

In [19]:
T.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Reading & transforming MIDI

We read MIDI-file, transform it into numpy-array, where **pitches follow columns, time runs along rows**.

Nonzero values of this array are the values of velocity of the pitch in the original composition.

Henceforth we first **element-wisely** take the root of degree 4 from elements of the matrix `T`, so that the matrix transforms the velocity:

$$\text{new velocity}(i)=\sum_{j} j \cdot (T^{1/4})_{ij}$$

Since MIDI-matrices encode velocity

$$M_{ij} = \text{velocity of j-th note at i-th time step} =  M_{ij}$$

the image of the MIDI-matrix is given by its multiplication from the right to the matrix $T^{1/4}$ (not its transpose):

$$M_{ik}^\text{new} = M_{ij}^\text{old}~(T^{1/4})_{jk} $$

In [20]:
from read_midi import Read_midi

from write_midi import write_midi

In [21]:
T_4th_root = T ** (1/4)

In [22]:
filename = 'handel_passacaigle.mid'

RM = Read_midi(song_path=filename,quantization=4)

m = RM.read_file()

In [23]:
for key in m.keys():
    m[key] = m[key] @ T_4th_root
    m[key] /= m[key].max()
    m[key] *= 127
    m[key] = np.round(m[key]).astype(int)
    
write_midi(m, ticks_per_beat=4, write_path='weighted_MIDI_transform_'+filename, tempo=80)